In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch.utils.data import random_split, DataLoader, TensorDataset
%matplotlib inline

import torch.optim as optim
import time
# import torchvision.models as models

In [2]:
train_ds= torchvision.datasets.CIFAR10(root="data/", train = True, transform=transforms.ToTensor(), download=True)
test_ds= torchvision.datasets.CIFAR10(root="data/", train = False, transform=transforms.ToTensor(), download=True)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
# len(train_ds) = 50000
# len(train_loader) = 391

127.8772378516624

In [3]:
batch_size=128
train_loader= DataLoader(train_ds, batch_size, shuffle =True)
test_leader=  DataLoader(test_ds, batch_size, shuffle = False)
# len(train_loader)

In [4]:
device=torch.device("cuda:1" if torch.cuda.is_available else "cpu")

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


num_classes=10
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
class Bottleneck(nn.Module):
    def __init__(self, inplane, planes, stride=1):
        super(Bottleneck,self).__init__()
        outplane=4*planes
        self.conv1 =   nn.Conv2d(inplane, planes,kernel_size=1,bias=False)
        self.batchN1 = nn.BatchNorm2d(planes)
        self.conv2 =   nn.Conv2d(planes, planes,kernel_size=3 ,stride=stride, padding=1 ,bias=False)
        self.batchN2 =  nn.BatchNorm2d(planes)
        self.conv3 =   nn.Conv2d(planes, outplane,kernel_size=1,bias=False)
        self.batchN3 =  nn.BatchNorm2d(outplane)
        
        self.shortcuts=nn.Sequential()
        if(outplane!=inplane or stride!=1):
            self.shortcuts = nn.Sequential(nn.Conv2d(inplane, outplane, kernel_size=1,stride=stride,bias=False) ,nn.BatchNorm2d(outplane))
            
    def forward(self, x):
        out=F.relu(self.batchN1(self.conv1(x)))
        out=F.relu(self.batchN2(self.conv2(out)))
        out=self.batchN3(self.conv3(out))
        out=F.relu(out+self.shortcuts(x))
        return out


In [6]:
class Resmodel(nn.Module):
    # def __init__(self, block=Bottleneck, no_bl=[3,4,6,3], output_no=10):
    def __init__(self, block, no_bl=[3,4,6,3], output_no=10):
        super(Resmodel,self).__init__()
        self.in_plane=64
        self.conv1= nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1 ,bias=False)
        self.batchN11= nn.BatchNorm2d(64)
        self.layer1= self._make_layer(block, 64, 3, 1)
        self.layer2= self._make_layer(block, 128, 4, 2)
        self.layer3= self._make_layer(block, 256, 6, 2)
        self.layer4= self._make_layer(block, 512, 3, 2)
        self.linear = nn.Linear(2048,10)
        
    def _make_layer(self, block, plane, numofblock, stride):
        layers=[]
        strides= [stride] + [1]*(numofblock-1)
        for st in strides:
            layers.append(Bottleneck(self.in_plane, plane, st))
            self.in_plane = plane*4
        return nn.Sequential(*layers)
    
    def forward(self, x):
        #size of x=[batchsize,3,32,32]
        out=self.conv1(x)           #after this operation size=[batchsize,64,32,32]
        out=self.batchN11(out)
        out=F.relu(out)
        # out= F.relu(self.batchN11(self.conv1(x)))
        out=self.layer1(out)        #after this operation the output size must be [batchsize,256,32,32]
        out=self.layer2(out)        #after this operation the output size must be [batchsize,512,16,16]
        out=self.layer3(out)        #after this operation the output size must be [batchsize,1024,8,8]
        out=self.layer4(out)        #after this operation the output size must be [batchsize,2048,4,4]
        out=F.avg_pool2d(out,4)     #after this operation the output size must be [batchsize,2048,1,1]
        out= out.view(out.size(0),-1)    #after this operation the output size must be [batchsize,1,2048]
        out=self.linear(out)         #after this operation the output size must be [batchsize,1,10]
        return out


In [7]:
model = Resmodel(Bottleneck)
model=model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(), lr= 0.01)


In [16]:
%%time
n_total_steps = len(train_loader)
# max_val_i=0
for epoch in range(4):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [128, 3, 32, 32]
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(max_val_i<i):
            max_val_i=i
        # if (i+1) % 20 == 0:
        #     print (f'Epoch [{epoch+1}/{4}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# print('max value of i=',max_val_i)
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)




Epoch [1/4], Step [200/391], Loss: 0.9331
Epoch [2/4], Step [200/391], Loss: 0.7526
Epoch [3/4], Step [200/391], Loss: 0.5657
Epoch [4/4], Step [200/391], Loss: 0.5027
max value of i= 390
Finished Training
CPU times: user 3min 47s, sys: 1min 28s, total: 5min 16s
Wall time: 5min 15s


In [52]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_leader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        # for i in range(batch_size):
        # print(images.shape)
        # break
        for i in range(16):
        # for i in range(128):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 70.84 %
Accuracy of plane: 73.5042735042735 %
Accuracy of car: 84.87394957983193 %
Accuracy of bird: 57.34265734265734 %
Accuracy of cat: 58.53658536585366 %
Accuracy of deer: 59.81308411214953 %
Accuracy of dog: 49.6124031007752 %
Accuracy of frog: 86.42857142857143 %
Accuracy of horse: 78.86178861788618 %
Accuracy of ship: 82.57575757575758 %
Accuracy of truck: 83.20610687022901 %


In [47]:
# len(test_leader) = 79
# len(images)
# 79*128 almost equals 10k
print(images.shape)

torch.Size([16, 3, 32, 32])


In [46]:


# y = torch.randn(1, 3, 4,4)
# out=F.avg_pool2d(y,4)
# print(out.shape)
# out= out.view(out.size(0),-1)
# print("new shape=",out.shape)


torch.Size([1, 3, 1, 1])
new shape= torch.Size([1, 3])
